../input/results/oxdna_random_6/L1/d-2-14/L1_d-2-14_2023-01-30-194808/L1_d-2-14_2023-01-30-194808/  
{'ratio_of_volume': 1.0, 'mean_volume': 6.4863029958777645, 'deviation_of_volume': 5.766281297335398}  
../input/results/oxdna_random_6/L1/d-7-15/L1_d-7-15_2023-01-30-063234/L1_d-7-15_2023-01-30-063234/  
{'ratio_of_volume': 0.95, 'mean_volume': 210.06664569316092, 'deviation_of_volume': 5.820491074083896}  

I would like to check this results  

In [14]:
ls measuring_volume

README.md              convexhull_volume.pyc  output_bonds.sh*
__pycache__/           get_conf_data.py       readers.py
base.py                get_top_data.py        readers.pyc
base.pyc               model.h                rna_model.h
convexhull_volume.py   output_bonds.py*       run_output_bonds.py


In [15]:
import sys
sys.path.append("measuring_volume")
import convexhull_volume as cv
import get_top_data as gtd

In [16]:
target_dir1 = "../input/results/oxdna_random_6/L1/d-2-14/L1_d-2-14_2023-01-30-194808//L1_d-2-14_2023-01-30-194808"
cv.convexhull_volume_all_strands(target_dir1)
cv.convexhull_volume_all_strands_meandev(target_dir1)
strands2particle, particle2strand = gtd.get_particle_strands_data(target_dir1)
print(float(len(strands2particle)))

../input/results/oxdna_random_6/L1/d-2-14/L1_d-2-14_2023-01-30-194808//L1_d-2-14_2023-01-30-194808 : mean volume is 6.4863029958777645
../input/results/oxdna_random_6/L1/d-2-14/L1_d-2-14_2023-01-30-194808//L1_d-2-14_2023-01-30-194808 : number of strands is 20.0
7.805962240200899
20.0
20.0


In [17]:
target_dir2 = "../input/results/oxdna_random_6/L1/d-7-15/L1_d-7-15_2023-01-30-063234/L1_d-7-15_2023-01-30-063234"
cv.convexhull_volume_all_strands(target_dir2)
cv.convexhull_volume_all_strands_meandev(target_dir2)
strands2particle, particle2strand = gtd.get_particle_strands_data(target_dir2)
print(float(len(strands2particle)))

../input/results/oxdna_random_6/L1/d-7-15/L1_d-7-15_2023-01-30-063234/L1_d-7-15_2023-01-30-063234 : mean volume is 210.06664569316092
../input/results/oxdna_random_6/L1/d-7-15/L1_d-7-15_2023-01-30-063234/L1_d-7-15_2023-01-30-063234 : number of strands is 19.0
4.096138463907446
19.0
19.0


In [18]:
import numpy as np
import matplotlib.pylab as plt
import sys
sys.path.append('../')
sys.path.append('.')
sys.path.append('measuring_volume/')
sys.path.append('common/')
import get_target_file as gtf
from mpl_toolkits.mplot3d import Axes3D
import get_top_data as gtd
import get_conf_data as gcd
from scipy.spatial import ConvexHull
# import k3d
import statistics
import math

def get_all_r(target_dir):
    conf_name = gtf.get_conf(target_dir)
    conf_f = open(conf_name, "r")
    col = 0


    x = []
    y = []
    z = []
    for l in conf_f:
        col += 1
        if col > 3:
            rx = float(l.split(" ")[0])
            ry = float(l.split(" ")[1])
            rz = float(l.split(" ")[2])
            x.append(rx)
            y.append(ry)
            z.append(rz)

    conf_f.close()
    return x, y, z

def translate_strand(points, box=100):
    # type of points is [[cm_pos_x, cm_pos_y, cm_pos_z]]
    # 2.translate(np.rint((s1.cm_pos - s2.cm_pos - diff1 + diff2) / self._box) * self._box)
    # diff1 = np.rint(s1.cm_pos / self._box ) * self._box

    for index, p in enumerate(points):
        if index == 0:
            plst = np.array(p)
            first_diff = np.rint(plst/box) * box
            first_p = plst
        else:
            plst = np.array(p)
            tmp_diff = np.rint(plst/box) * box
            p += np.rint((first_p - plst - first_diff + tmp_diff) / box) * box




def get_r(target_dir, strands):

    target_strands = list(strands)
    conf_dic = gcd.get_conf_data(target_dir)


    x = []
    y = []
    z = []

    for particle_id in target_strands:
        x.append(conf_dic[particle_id]["rx"])
        y.append(conf_dic[particle_id]["ry"])
        z.append(conf_dic[particle_id]["rz"])
    
    return x, y, z

def convexhull_volume(x, y, z):
    points = []
    for i in range(len(x)):
        lst = [x[i], y[i], z[i]]
        points.append(lst)
    
    print(points)
    # 追加
#     points = np.mod(points, 100)
    box = 100
    
    hull = ConvexHull(points)

    points = np.array(points)



    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")

    # Plot defining corner points
    ax.plot(points.T[0], points.T[1], points.T[2], "ko")
    # ax.plot(x, y, z, "ko")

    # 12 = 2 * 6 faces are the simplices (2 simplices per square face)
    for s in hull.simplices:
        s = np.append(s, s[0])  # Here we cycle back to the first coordinate
        ax.plot(points[s, 0], points[s, 1], points[s, 2], "r-")
        # ax.plot(x, y, z, "r-")

    # Make axis label
    for i in ["x", "y", "z"]:
        eval("ax.set_{:s}label('{:s}')".format(i, i))

    plt.show()

    
    return hull.volume
    
def plot(X, Y, Z, target_dir):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(X, Y, Z)
    plt.show()

def convexhull_volume_all_strands(target_dir):
    strands2particle, particle2strand = gtd.get_particle_strands_data(target_dir)

    volumes = {}

    for strand in strands2particle:
        x, y, z = get_r(target_dir, strands2particle[strand])
        volumes[strand] = convexhull_volume(x, y, z)
        # plot(x, y, z, target_dir)
    
    mean_volume = 0.0
    num_of_strands = float(len(strands2particle))

    for strand in volumes:
        # print(volumes[strand])
        mean_volume += volumes[strand]
    
    mean_volume /= num_of_strands
    print(target_dir + " : mean volume is " + str(mean_volume))
    print(target_dir + " : number of strands is " + str(num_of_strands))
    return mean_volume

def convexhull_volume_all_strands_meandev(target_dir):
    strands2particle, particle2strand = gtd.get_particle_strands_data(target_dir)

    volumes = {}

    for strand in strands2particle:
        x, y, z = get_r(target_dir, strands2particle[strand])
        volumes[strand] = convexhull_volume(x, y, z)
        #         print(x, y, z)
        # plot(x, y, z, target_dir)
    
    print(volumes)
    
    mean_volume = 0.0
    num_of_strands = float(len(strands2particle))

    for strand in volumes:
        mean_volume += volumes[strand]
    
    mean_volume /= num_of_strands
    dev_volume = statistics.pstdev(volumes)
    print(num_of_strands)

    return mean_volume, dev_volume




In [19]:
convexhull_volume_all_strands_meandev(target_dir2)

[[167.491879563092, -9.40455397662559, 24.5214205372287], [166.824905465035, -8.93224617621311, 24.6619086284744], [166.374868514815, -9.06303413587304, 24.9572867034575], [166.676843174772, -8.5347512152448, 24.9734622062313], [167.131223978396, -8.40213015383009, 25.1296545566318], [167.445448237781, -8.31897626900663, 25.4524939964067], [166.714901400508, -8.01600732084819, 25.9080930739791], [166.896727566488, -7.45701218501478, 25.6699341224976], [167.123067305771, -6.99074145621556, 25.6028492802924], [167.554539601408, -6.74481220088791, 25.4738563964714], [168.050295235092, -6.66976918610344, 25.3034421832236], [167.939657803121, -6.17301500428569, 24.4261079587713], [167.952128827766, -6.48170018061958, 24.3908092124593], [168.111280016005, -6.93407976392092, 24.5424262988704], [168.109885223079, -7.37151833516957, 24.8620945464866], [168.190142409892, -7.69795672689818, 25.3333811052954], [168.067416054982, -7.72196989372388, 25.9120523354281], [167.72539140598, -7.7075014859

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [39]:
lst = [[96.6518314888424, -227.392072286216, 219.803277275761],
 [96.5452072329311, -227.190120531322, 220.24401646738],
 [96.8502487237835, -226.135694873475, 219.755073482845],
 [97.2485868444429, -226.147834659714, 219.424198893554],
 [97.329554600845, -225.853807714041, 218.991817076538],
 [97.7254510103363, -225.514993502087, 218.737878709592],
 [98.0387206841756, -225.110212797437, 218.767988172248],
 [98.4988166730302, -224.95130673137, 218.892231064315],
 [99.0468681108471, -224.851025756903, 218.895433328301],
 [99.6603193990761, -224.961411671824, 218.881514324423],
 [100.142956678409, -225.081177100267, 218.636170273363],
 [100.517575856033, -225.051403140247, 218.152239399471],
 [100.606870364209, -224.992508337628, 217.652953615376],
 [101.252718485089, -225.691310714448, 217.463493543791],
 [101.585519804945, -225.533005823739, 217.651442825199],
 [101.846358484485, -225.213175813547, 217.816470964865],
 [101.930490128441, -224.740750351131, 217.94069230943],
 [102.139422792698, -224.620638895751, 218.381344942305],
 [102.368259748101, -224.755885095941, 218.849774895617],
 [102.324052147393, -223.939748652477, 219.53022559266],
 [199.489127807409, 177.220808914266, 316.383838769977],
 [198.777119094293, 176.242769436341, 316.923143971852],
 [198.995259372871, 176.26198951281, 317.370476337845],
 [199.502056703995, 176.972160656934, 317.864336867841],
 [199.878433058766, 176.356885453373, 317.245736012956],
 [199.65160661812, 175.898520831043, 318.060671787543],
 [199.430723526655, 175.525841974302, 317.918502347712],
 [199.273338697072, 175.049998742355, 317.802915156943],
 [199.155997725213, 174.471739515377, 317.989162539842],
 [198.991330351767, 174.086298963222, 318.340303285739],
 [198.771725982412, 173.928405694356, 318.79420194709],
 [198.594270764397, 173.980029526168, 319.30075588847],
 [198.328653971525, 174.209619595546, 319.644163950024],
 [198.04110251874, 174.624712490993, 319.733820416899],
 [197.57951651398, 174.859503274299, 319.928263640322],
 [197.060257648537, 174.948437588954, 319.840396646307],
 [196.642961450098, 174.799607949156, 320.042225894988],
 [196.351150974153, 174.486514451996, 320.266276369638],
 [196.249676880215, 174.261128995387, 320.679837359277],
 [196.255869548483, 174.596328359795, 321.207938340279]]

In [33]:
np.mod(lst, 100)

array([[96.65183149, 72.60792771, 19.80327728],
       [96.54520723, 72.80987947, 20.24401647],
       [96.85024872, 73.86430513, 19.75507348],
       [97.24858684, 73.85216534, 19.42419889],
       [97.3295546 , 74.14619229, 18.99181708],
       [97.72545101, 74.4850065 , 18.73787871],
       [98.03872068, 74.8897872 , 18.76798817],
       [98.49881667, 75.04869327, 18.89223106],
       [99.04686811, 75.14897424, 18.89543333],
       [99.6603194 , 75.03858833, 18.88151432],
       [ 0.14295668, 74.9188229 , 18.63617027],
       [ 0.51757586, 74.94859686, 18.1522394 ],
       [ 0.60687036, 75.00749166, 17.65295362],
       [ 1.25271849, 74.30868929, 17.46349354],
       [ 1.5855198 , 74.46699418, 17.65144283],
       [ 1.84635848, 74.78682419, 17.81647096],
       [ 1.93049013, 75.25924965, 17.94069231],
       [ 2.13942279, 75.3793611 , 18.38134494],
       [ 2.36825975, 75.2441149 , 18.8497749 ],
       [ 2.32405215, 76.06025135, 19.53022559],
       [99.48912781, 77.22080891, 16.383

In [22]:
lst = [99.6603194 , 75.03858833, 18.88151432]
lst = np.array(lst)
print(np.rint(lst/100) * 100)

[100. 100.   0.]
